# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#%matplotlib notebook

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

#Configure gmaps
gmaps.configure(api_key=g_key)
#print(g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_wx_df = pd.read_csv("../output_data/cities_weather.csv")
city_wx_df

,City,Country,City ID,lon,lat,temperature,max temperature,humidity,clouds,windspeed
0,Esmeraldas,BR,3464008,-44.3139,-19.7625,79.39,80.60,61,40,11.50
1,Najrān,SA,103630,44.1277,17.4924,78.80,78.80,25,40,11.50
2,Busselton,AU,2075265,115.3333,-33.6500,61.61,62.01,77,0,1.99
3,New Norfolk,AU,2155415,147.0587,-42.7826,56.32,57.00,73,3,3.09
4,Mar del Plata,AR,3430863,-57.5575,-38.0023,76.59,78.01,38,0,17.27
...,...,...,...,...,...,...,...,...,...,...
563,Mabaruma,GY,3377301,-59.7833,8.2000,79.30,79.30,88,100,5.68
564,Aksu,CN,1529660,80.2644,41.1231,41.05,41.05,60,94,1.19
565,West Bay,KY,3580477,-81.4167,19.3667,81.84,82.99,65,20,17.27
566,Paramonga,PE,3933024,-77.8333,-10.6667,74.93,74.93,63,5,10.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
city_list = city_wx_df['City'].tolist()
humidity_rate = city_wx_df['humidity'].astype(float)

marker_locations = city_wx_df[['lat', 'lon']]

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
                            info_box_content=[f'Ideal {city}' for city in city_list])
heat_layer = gmaps.heatmap_layer(marker_locations, weights=humidity_rate,
                               dissipating=False, max_intensity=100,
                               point_radius=1)
#fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find ideal weather condition
# Max temperature lower than 80 degrees but higher than 70
ideal_df = city_wx_df[city_wx_df['max temperature'] < 80]
ideal_df = ideal_df[ideal_df['max temperature'] > 70]

# Wind speed less than 10 mph
ideal_df = ideal_df[ideal_df['windspeed'] < 10]

# Zero cloudiness
ideal_df = ideal_df[ideal_df['clouds'] == 0]

ideal_df.dropna()

ideal_df

,City,Country,City ID,lon,lat,temperature,max temperature,humidity,clouds,windspeed
7,Rikitea,PF,4030556,-134.9692,-23.1203,79.00,79.00,69,0,9.35
8,Geraldton,AU,2070998,114.6000,-28.7667,71.60,71.60,83,0,4.61
46,Pisco,PE,3932145,-76.2167,-13.7000,72.63,73.99,78,0,9.22
287,Ubari,LY,2219235,12.7805,26.5921,71.65,71.65,17,0,8.95
426,Dahegām,IN,1273704,72.8167,23.1667,74.73,74.73,18,0,4.74
472,Acajutla,SV,3587498,-89.8275,13.5928,71.60,71.60,56,0,2.30


In [6]:
ideal_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 7 to 472
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City             6 non-null      object 
 1   Country          6 non-null      object 
 2   City ID          6 non-null      int64  
 3   lon              6 non-null      float64
 4   lat              6 non-null      float64
 5   temperature      6 non-null      float64
 6   max temperature  6 non-null      float64
 7   humidity         6 non-null      int64  
 8   clouds           6 non-null      int64  
 9   windspeed        6 non-null      float64
dtypes: float64(5), int64(3), object(2)
memory usage: 528.0+ bytes


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
hotel_df = ideal_df[['City','Country','lat','lon']]
hotel_df['Hotel Name'] = ""
hotel_df['Rating'] = ""
hotel_df.reset_index(level=None,drop=True,inplace=True)
hotel_df
 

C:\Users\Sharon\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Sharon\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Country,lat,lon,Hotel Name,Rating
0,Rikitea,PF,-23.1203,-134.9692,,
1,Geraldton,AU,-28.7667,114.6000,,
2,Pisco,PE,-13.7000,-76.2167,,
3,Ubari,LY,26.5921,12.7805,,
4,Dahegām,IN,23.1667,72.8167,,
5,Acajutla,SV,13.5928,-89.8275,,


In [24]:
city_list = hotel_df['City']

city_list

0      Rikitea
1    Geraldton
2        Pisco
3        Ubari
4      Dahegām
5     Acajutla
Name: City, dtype: object

In [26]:
#
#coords = "-33.9180,25.5701"
target_type = 'lodging'
radius = 5000
# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#city_list = ['Port Elizabeth']   #, 'New Norfolk']
#coords = '-33.9180,25.5701'    #,-42.7826	147.0587]
nearest_hotels = []
location_list = []
hotel_ratings = []
heat_map_city_list = []
heat_map_locations = []
i = 0
#for city in city_list:

for index, row in  hotel_df.iterrows():
    #print(row['City'])
    #print('*********************************************************')
    #row = hotel_df.loc[hotel_df['City'] == city]
    ##print(row)
    
    latitude = row['lat']
    longitude = row['lon']
    #i = i+1
    coords = f"{latitude},{longitude}"
    location_list.append(coords)
    
    #print (coords)
    params = {
        'location':coords,
        'types': target_type,
        'radius': radius,
        'key': g_key
    }
    
    hotels = requests.get(base_url, params).json()
    
    #print(json.dumps(hotels, indent=4, sort_keys=True))
    
    
    
    
    try:
        #hotel_name = hotels['results'][0]['name']
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
        #rating = hotels['results'][0]['rating']
        hotel_df.loc[index,'Rating'] = hotels['results'][0]['rating']
        #if hotel_name != None:
        #heat_map_city_list.append(city)
        #heat_map_locations.append(coords)
            
        #rating = hotels['results'][0]['rating']
        #nearest_hotels.append(hotel_name)
        #hotel_ratings.append(rating)
    except:
        print(f'EXCEPTION:{hotel_name} is missing ?')
    #if (rating != None):
    #    hotel_ratings.append(rating)
    #else:
    #    hotel_ratings.append(0)
    #print(hotel_name)
    #print(rating)
    
hotel_df

C:\Users\Sharon\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


EXCEPTION:asi es is missing ?


,City,Country,lat,lon,Hotel Name,Rating
0,Rikitea,PF,-23.1203,-134.9692,Pension Maro'i,4
1,Geraldton,AU,-28.7667,114.6000,Broadwater Mariner Resort,4
2,Pisco,PE,-13.7000,-76.2167,Hotel San Isidro Oficial,3.9
3,Ubari,LY,26.5921,12.7805,فندق أوباري,2.8
4,Dahegām,IN,23.1667,72.8167,Malhaar Resort,3.7
5,Acajutla,SV,13.5928,-89.8275,asi es,


In [ ]:
hotel_df.info()

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# I can't get this line of code to work, so I am going back to what worked for me earlier
locations = hotel_df[["lat", "lon"]]




In [ ]:

    


# Add marker layer ontop of heat map
#hotel_list = hotel_df['Hotel Name'].tolist()

fig = gmaps.figure()
hotel_markers = gmaps.marker_layer(locations,hotel_info)
heat_layer = gmaps.heatmap_layer(locations, weights=None,
                                dissipating=False, max_intensity=100,
                                point_radius=1)
fig.add_layer(heat_layer)
fig

# Display figure
